In [1]:
from bs4 import BeautifulSoup as bs

from time import sleep

from selenium import webdriver

from selenium.webdriver.chrome.service import Service

# ChromeDriver 的下載管理工具
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 下載檔案的工具
import wget

from selenium.webdriver.support.ui import Select

from fake_useragent import UserAgent

import re

In [2]:
ua = UserAgent()
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         # 最大化視窗
my_options.add_argument("--incognito")               # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")   # 取消通知
my_options.add_argument(f'--user-agent={ua.random}') # (Optional)加入 User-Agent


# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'report')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

    
#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})


# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)



In [3]:
# 開啟 交通部觀光局觀光統計資料庫月表
driver.get("https://stat.taiwan.net.tw/statistics?action=month")

# 取得檢視原始碼的內容 (page_source 取得的 html，是動態的、使用者操作過後的結果)
html = driver.page_source

# 指定 lxml 作為解析器
soup = bs(html, "lxml")

# 建立要爬取的網址清單
url_data = []

regex01=r'[\u4e00-\u9fa5]+'
regex02=r'(?<=[\u4e00-\u9fa5])[a-zA-Z\s]+'

for a in soup.select('tbody > tr > td > a'):
    # b=a.get_text()
    title = re.search(regex01, a.get_text())
    url_data.append({
        
        #match[0]是regex所代表的整個完整比對的字串
        'title': title[0],
        'link' :"https://stat.taiwan.net.tw"+a.get("href")})

driver.quit()
url_data


[{'title': '來臺旅客居住地分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/residence'},
 {'title': '來臺旅客國籍分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/nationality'},
 {'title': '來臺旅客性別及來臺目的分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/gender/purpose'},
 {'title': '來臺旅客年齡分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/age'},
 {'title': '來臺旅客搭乘交通工具及入境港口分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/transport'},
 {'title': '來臺旅客停留夜數統計',
  'link': 'https://stat.taiwan.net.tw/statistics/month/inbound/lengthOfStay'},
 {'title': '中華民國國民出國目的地分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/outbound/destination'},
 {'title': '中華民國國民出國性別及年齡分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/outbound/gender/age'},
 {'title': '中華民國國民出國停留夜數分析',
  'link': 'https://stat.taiwan.net.tw/statistics/month/outbound/lengthOfStay'},
 {'title': '搭乘郵輪來臺旅客人數',
  'link': 'https://stat.taiwan.net.t

In [4]:
#建立要爬取的年份清單
year = []
for rocyear,adyear in zip( range(97,113),range(2008,2024) ):
    year.append(f'{rocyear}({adyear})')

#建立要爬取的月份清單
month = []
for mm in range(1,13):
    month.append(f'{mm}月')
    
print(year) 
print(month)

['97(2008)', '98(2009)', '99(2010)', '100(2011)', '101(2012)', '102(2013)', '103(2014)', '104(2015)', '105(2016)', '106(2017)', '107(2018)', '108(2019)', '109(2020)', '110(2021)', '111(2022)', '112(2023)']
['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']


In [8]:

for index, obj in enumerate(url_data):

   # 建立下載路徑/資料夾，不存在就新增，資料夾名稱為url_data的title
    # url_data[index]['title']}
    folderPath = os.path.join(os.getcwd(), f"report\{url_data[index]['title']}")
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    

    #預設下載路徑
    my_options.add_experimental_option("prefs", {"download.default_directory": folderPath})
    driver = webdriver.Chrome(
        options=my_options,
        service=Service(ChromeDriverManager().install())
    )

    sleep(1)

    # 開啟 交通部觀光局觀光統計資料庫月表，用迴圈跑12個網址 ，但"觀光遊憩據點遊客人次單月明細表"這個會抓不到，因為他只有單月的選項
    driver.get(url_data[index]['link'])
    sleep(1)


    for y in year: 
        try:
            #選擇年份
            yy = Select(driver.find_element(By.CSS_SELECTOR, 'div.col-md-3 > select'))
            yy.select_by_visible_text(y)
        
            for m in month:
                try:
                    #選擇月份
                    end_month = Select(driver.find_element(By.CSS_SELECTOR, 'div > select[formcontrolname="monthEnd"]'))
                    end_month.select_by_visible_text(m)
            
                    start_month = Select(driver.find_element(By.CSS_SELECTOR, 'div > select[formcontrolname="monthStart"]'))
                    start_month.select_by_visible_text(m)
                    
                    sleep(0.1)
                    
                    #按下excel產出(Export)
                    driver.find_element(
                    By.CSS_SELECTOR, 
                    'div.col-md-7 > input').click()
                    
                    print(f"下載 {url_data[index]['title']}{y}年{m}報表")
                          
                    sleep(3)

                #因為2023的資料只會到3月，故會有Exception，用continue跳過，繼續迴圈
                except NoSuchElementException:
                    continue 

            # 讓start_month回到一月份，這樣end_month才有選項可以選
            start_month = Select(driver.find_element(By.CSS_SELECTOR, 'div > select[formcontrolname="monthStart"]'))
            start_month.select_by_visible_text('1月')
            sleep(1)

        #因為不是每張報表都有2008~2023年的資料
        except NoSuchElementException:
            continue 

print("全部下載完成!")
driver.quit()


下載 來臺旅客居住地分析97(2008)年1月報表
下載 來臺旅客居住地分析97(2008)年2月報表
下載 來臺旅客居住地分析97(2008)年3月報表
下載 來臺旅客居住地分析97(2008)年4月報表
下載 來臺旅客居住地分析97(2008)年5月報表
下載 來臺旅客居住地分析97(2008)年6月報表
下載 來臺旅客居住地分析97(2008)年7月報表
下載 來臺旅客居住地分析97(2008)年8月報表
下載 來臺旅客居住地分析97(2008)年9月報表
下載 來臺旅客居住地分析97(2008)年10月報表
下載 來臺旅客居住地分析97(2008)年11月報表
下載 來臺旅客居住地分析97(2008)年12月報表
下載 來臺旅客居住地分析98(2009)年1月報表
下載 來臺旅客居住地分析98(2009)年2月報表
下載 來臺旅客居住地分析98(2009)年3月報表
下載 來臺旅客居住地分析98(2009)年4月報表
下載 來臺旅客居住地分析98(2009)年5月報表
下載 來臺旅客居住地分析98(2009)年6月報表
下載 來臺旅客居住地分析98(2009)年7月報表
下載 來臺旅客居住地分析98(2009)年8月報表
下載 來臺旅客居住地分析98(2009)年9月報表
下載 來臺旅客居住地分析98(2009)年10月報表
下載 來臺旅客居住地分析98(2009)年11月報表
下載 來臺旅客居住地分析98(2009)年12月報表
下載 來臺旅客居住地分析99(2010)年1月報表
下載 來臺旅客居住地分析99(2010)年2月報表
下載 來臺旅客居住地分析99(2010)年3月報表
下載 來臺旅客居住地分析99(2010)年4月報表
下載 來臺旅客居住地分析99(2010)年5月報表
下載 來臺旅客居住地分析99(2010)年6月報表
下載 來臺旅客居住地分析99(2010)年7月報表
下載 來臺旅客居住地分析99(2010)年8月報表
下載 來臺旅客居住地分析99(2010)年9月報表
下載 來臺旅客居住地分析99(2010)年10月報表
下載 來臺旅客居住地分析99(2010)年11月報表
下載 來臺旅客居住地分析99(2010)年12月報表
下載 來臺旅客居住地分析100(2011)年1月報表
下載 來臺旅客居住地分析100(2011)年2月報表
下

In [7]:
#"觀光遊憩據點遊客人次單月明細表"只有單月的選項，所以要另外抓

folderPath = os.path.join(os.getcwd(), f"report\{url_data[10]['title']}")
if not os.path.exists(folderPath):
    os.makedirs(folderPath)


my_options.add_experimental_option("prefs", {"download.default_directory": folderPath})
driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install())
)

driver.get("https://stat.taiwan.net.tw/statistics/month/scenicSpot/single")
sleep(1)


for y in year: 
    try:
        yy = Select(driver.find_element(By.CSS_SELECTOR, 'div.col-md-3 > select'))
        yy.select_by_visible_text(y)
    
        for m in month:
            try:
                mm2 = Select(driver.find_element(By.CSS_SELECTOR, 'div > select[formcontrolname="monthEnd"]'))
                mm2.select_by_visible_text(m)
        
                sleep(0.1)
                
                driver.find_element(
                By.CSS_SELECTOR, 
                'div.col-md-7 > input').click()

                print(f"下載 {url_data[10]['title']}{y}年{m}報表")

                sleep(3)
            except NoSuchElementException:
                continue 

    except NoSuchElementException:
        continue 
    
print("全部下載完成!")
driver.quit()


下載 觀光遊憩據點遊客人次單月明細表97(2008)年1月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年2月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年3月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年4月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年5月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年6月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年7月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年8月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年9月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年10月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年11月報表
下載 觀光遊憩據點遊客人次單月明細表97(2008)年12月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年1月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年2月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年3月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年4月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年5月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年6月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年7月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年8月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年9月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年10月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年11月報表
下載 觀光遊憩據點遊客人次單月明細表98(2009)年12月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年1月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年2月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年3月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年4月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年5月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年6月報表
下載 觀光遊憩據點遊客人次單月明細表99(2010)年7月報表
下載